In [1]:
import os
import sys

BOOTPATH = os.path.dirname(sys.path[0])
sys.path.insert(0, BOOTPATH)
import dpeeg
print(dpeeg.__version__)

0.3.1


In [2]:
from dpeeg.data import transforms

trans = transforms.ComposeTransforms(
    transforms.Normalization(),
    transforms.Unsqueeze(),
)

trans

ComposeTransforms(
 (0): Normalization(mode=z-score)
 (1): Unsqueeze(dim=1)
)